In [3]:
import torch
import joblib
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder

def preprocess_text(text, vectorizer):
    # Aplicar a mesma transformação utilizada no conjunto de treinamento
    text_transformed = vectorizer.transform([text])
    tensor = torch.Tensor(text_transformed.toarray())
    return tensor

len_entrada = 25659
len_saida = 5

le = LabelEncoder()
le.fit_transform(
    ["ARISTOTLE",
    "NIETZSCHE",
    "PLATO",
    "SCHOPENHAUER",
    "THOMAS AQUINAS"]
)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()

        self.linear_relu_stack = nn.Sequential(
            nn.Linear(len_entrada, 80),
            nn.ReLU(),
            nn.Linear(80, 80),
            nn.ReLU(),
            nn.Linear(80, len_saida)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

# Carregar o vetorizador
loaded_vectorizer = joblib.load(f"./modelos/vetorizador_{len_entrada}.pkl")

device = 'mps' if torch.backends.mps.is_available() else 'cpu'
# Carregar o modelo
loaded_model = NeuralNetwork().to(device)
loaded_model.load_state_dict(torch.load(f"./modelos/modelo_{len_entrada}.pth"))
loaded_model.eval()


input_text = input("Digite uma frase: ")
# Utilizar o modelo carregado para fazer previsões
preprocessed_text = preprocess_text(input_text, loaded_vectorizer)
preprocessed_text = preprocessed_text.to(device)

with torch.no_grad():
    loaded_model.eval()
    prediction = loaded_model(preprocessed_text)
    predicted_class = prediction.argmax().item()

predicted_class_label = le.inverse_transform([predicted_class])[0]

print("Frase:", input_text)
print("Classe prevista:", predicted_class_label)

Frase: God is good
Classe prevista: THOMAS AQUINAS
